# ResNet

## 残差块（ResNet Block）

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

/Users/apple/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


- ResNet沿用了VGG全3x3卷积层的设计，残差块里首先有2个有相同输出通道数的3x3卷积层。每个卷积层后接一个批量归一化（Batch Norm）和ReLU激活函数。然后我们将输入跳过这2个卷积运算后直接加在最后的ReLU激活函数前。

- 这样的设计要求2个卷积层的输出与输入形状一样，从而可以相加。如果想要改变通道数，就需要引入一个额外的1x1卷积层来将输入变换成需要的形状后再做相加运算。

In [7]:
class Residual(nn.Module):
    def __init__(self, input_channels, num_channels, use_1x1conv=False, strides=1):
        ####
        #@param
        # input_channels:输入通道数
        # num_channels:  输出通道数
        # use_1x1conv:   是否使用1x1卷积层
        # strides:       步幅
        ####
        super().__init__()
        #第一个3x3卷积，输入和输出的通道数相同，也支持通过选择不同的stride（步幅）来调整通道数。如果选择了调整通道数，1x1卷积层中也会做相同变换
        self.conv1 = nn.Conv2d(
            input_channels, num_channels, kernel_size=3,
                            padding=1, stride=strides)
        
        self.conv2 = nn.Conv2d(
            num_channels, num_channels, kernel_size=1, padding=1)
        
        if use_1x1conv:
            #注意这里的“输入通道数”和“输出通道数”，输出通道数为num_channels从而和3x3卷积层的输出结果保持通道数相同 --> 从而能做相加运算
            self.conv3 = nn.Conv2d(input_channels, num_channels, kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv(Y))
        if self.conv3:
            X = self.conv3(X)
        return F.relu(Y + X)